In [ ]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-1wvb47ec/gym-walk_6fd04ee310894f328a7b2c7e57400d10
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-1wvb47ec/gym-walk_6fd04ee310894f328a7b2c7e57400d10
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=4053 sha256=27fc0311d35d5d5e74b42b38cd319bc65f1f4cb4b11e7bb048d6d4f4ea732c80
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ym5b3zi/wheels/24/fe/c4/0cbc7511d29265bad7e28a09311db3f87f0cafba74af54d530
Successfully built gym-walk


In [ ]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [ ]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [ ]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

**Creating the Frozen Lake environment**

In [ ]:
envdesc = ['GFFF','HFFF','FHFF','HFSF']
env = gym.make('FrozenLake-v1',desc=envdesc)
init_state = env.reset()
goal_state = 0
P = env.env.P

In [ ]:
P

{0: {0: [(1.0, 0, 0, True)],
  1: [(1.0, 0, 0, True)],
  2: [(1.0, 0, 0, True)],
  3: [(1.0, 0, 0, True)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 1.0, True),
   (0.3333333333333333, 5, 0.0, False)],
  1: [(0.3333333333333333, 0, 1.0, True),
   (0.3333333333333333, 5, 0.0, False),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 1.0, True)]},
 2: {0: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 6, 0.0, False)],
  1: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 6, 0.0, False),
   (0.3333333333333333, 3, 0.0, False)],
  2: [(0.3333333333333333, 6, 0.0, False),
   (0.3333333333333333, 3, 0.0, False),
   (0.3333333333333333, 2, 0.0, False)],
  3: [(0.33333333333

**Value Iteration Algorithm**


In [ ]:
def value_iteration(P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
      Q=np.zeros((len(P),len(P[0])),dtype=np.float64)
      for s in range(len(P)):
        for a in range(len(P[s])):
          for prob,next_state,reward,done in P[s][a]:
            Q[s][a]+=prob*(reward+gamma*V[next_state]*(not done))
      if(np.max(np.abs(V-np.max(Q,axis=1))))<theta:
        break
      V=np.max(Q,axis=1)
    pi=lambda s:{s:a for s , a in enumerate(np.argmax(Q,axis=1))}[s]
    return V, pi

In [ ]:
#Finding the optimal policy
V_best_v, pi_best_v = value_iteration(P, gamma=0.80)


In [ ]:

# Printing the policy
print("HARINI V")
print("212222230044")
print('Optimal policy and state-value function (VI):')
print_policy(pi_best_v, P)

HARINI V
212222230044
Optimal policy and state-value function (VI):
Policy:
|           | 01      ^ | 02      < | 03      ^ |
|           | 05      > | 06      ^ | 07      ^ |
| 08      < |           | 10      > | 11      ^ |
|           | 13      v | 14      > | 15      < |


In [ ]:
# printing the success rate and the mean return
print("HARINI V")
print("212222230044")
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_v, goal_state=goal_state)*100,
    mean_return(env, pi_best_v)))

HARINI V
212222230044
Reaches goal 69.00%. Obtains an average undiscounted return of 0.6900.


In [ ]:
# printing the state value function
print("HARINI V")
print("212222230044")
print_state_value_function(V_best_v, P, prec=4)

HARINI V
212222230044
State-value function:
|           | 01 0.5457 | 02 0.2507 | 03 0.1433 |
|           | 05 0.1838 | 06 0.1437 | 07 0.1043 |
| 08    0.0 |           | 10 0.0644 | 11 0.0614 |
|           | 13 0.0132 | 14 0.0363 | 15 0.0355 |
